## Weekly mileage recorded versus 10% increase from 25 miles at 14 Oct 2024


In [5]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio
pio.renderers.default = 'iframe'  # This ensures the plot displays in Jupyter

# Step 1: Read in the data
file_path = 'Activities.csv'  # Assuming the file is in the current working directory
data = pd.read_csv(file_path)

# Step 2: Process the data
# Convert 'Date' to datetime and clean up data
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data.replace('--', pd.NA, inplace=True)

# Convert necessary columns to numeric
data['Distance'] = pd.to_numeric(data['Distance'], errors='coerce')
data['Total Time'] = pd.to_timedelta(data['Total Time'], errors='coerce')
data['Avg HR'] = pd.to_numeric(data['Avg HR'], errors='coerce')

# Filter for running activities only
running_data = data[data['Activity Type'].isin(['Running', 'Trail Running'])]

# Step 3: Create a new column for the week and group by week
running_data['Week'] = running_data['Date'].dt.isocalendar().week
recent_running_data = running_data[running_data['Date'] >= '2024-08-01']

# Aggregate weekly mileage
weekly_mileage = recent_running_data.groupby('Week')['Distance'].sum().reset_index()

# Step 4: Define the variables for the interactive visualisation
# Start projection from 25 miles after the injury prevention period
new_starting_mileage = 25
weeks_to_project = np.arange(weekly_mileage['Week'].max() + 1, 10 + 53)  # Project until week 10 of 2025
new_projected_mileage = [new_starting_mileage]
for i in range(1, len(weeks_to_project)):
    new_projected_mileage.append(new_projected_mileage[-1] * 1.10)

# Create a date range starting from the first week of August 2024 for plotting
start_date = pd.to_datetime('2024-08-01')
week_dates = pd.date_range(start=start_date, periods=len(weekly_mileage) + len(weeks_to_project), freq='W-MON')

# Manually define the drop period for injury prevention
drop_start = week_dates[len(weekly_mileage) - 2]
drop_end = week_dates[len(weekly_mileage)]

# Define the Winter Green Man Ultra race date
race_date = pd.to_datetime('2025-03-01')

# Step 5: Create the Plotly interactive visualisation
fig = go.Figure()

# Plot actual mileage
fig.add_trace(go.Scatter(x=week_dates[:len(weekly_mileage)], 
                         y=weekly_mileage['Distance'], 
                         mode='lines+markers', 
                         name='Actual Mileage', 
                         line=dict(color='green')))

# Plot projected mileage
fig.add_trace(go.Scatter(x=week_dates[len(weekly_mileage):], 
                         y=new_projected_mileage, 
                         mode='lines+markers', 
                         name='New Projected 10% Increase', 
                         line=dict(dash='dash', color='red')))

# Add injury prevention period
fig.add_shape(type="rect",
              x0=drop_start, x1=drop_end, y0=0, y1=max(weekly_mileage['Distance']),
              fillcolor="lightblue", opacity=0.3, line_width=0, name='Injury Prevention Period')

# Add annotation for injury prevention
fig.add_annotation(x=drop_start, y=5, text="Injury Prevention Period", showarrow=False, font=dict(color="black"))

# Add Winter Green Man Ultra race date
fig.add_vline(x=race_date, line=dict(color='purple', dash='dash'), name='Winter Green Man Ultra')
fig.add_annotation(x=race_date, y=max(new_projected_mileage)*0.8, 
                   text="Winter Green Man Ultra", showarrow=False, textangle=90, font=dict(color="purple"))

# Update layout
fig.update_layout(title='Weekly Running Mileage with New 10% Increase Projection (Miles)',
                  xaxis_title='Week Starting Date',
                  yaxis_title='Total Distance (miles)',
                  xaxis_tickformat='%Y-%m-%d',
                  xaxis_tickangle=-45,
                  hovermode="x unified")

# Show the interactive plot
# fig.show()
fig.write_html("weekly_mileage.html")



/var/folders/4n/18wn5fm9177dpn28hhj4f8w00000gn/T/ipykernel_91563/3304019837.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

